# Sample Simulation


## Setup

In [ ]:
# Install the RocketPy package using pip
!pip install rocketpy

In [ ]:
# Import necessary classes from the RocketPy library

# Environment: Represents the atmospheric and environmental conditions during flight
# SolidMotor: Represents a solid rocket motor with specified characteristics
# Rocket: Represents a rocket with specified design parameters and components
# Flight: Represents a flight simulation of a rocket in a given environment and with specified motor and rocket configurations

from rocketpy import Environment, SolidMotor, Rocket, Flight


In [ ]:
# Used for tidying up the outputted data - not critical for operations

# Configure the InlineBackend to display SVG figures
%config InlineBackend.figure_formats = ['svg']

# Ensure that matplotlib plots are displayed inline in the Jupyter Notebook
%matplotlib inline


## Setting Up a Simulation


### Creating an Environment


In [ ]:
# Create an Environment object representing the atmospheric and environmental conditions during flight
env = Environment(latitude=55.43753666449806, longitude=-5.687982477918211, elevation=13)


In [ ]:
import datetime

# Get the date of tomorrow
# Can be any day, just change the 'days' variable equal to the unmber of days between today and the day you want
tomorrow = datetime.date.today() + datetime.timedelta(days=1)

# Set the date and time in the Environment object, with the hour given in UTC time
env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))


In [ ]:
# Set the atmospheric model type to "Forecast" and specify the file for the Global Forecast System (GFS)
env.set_atmospheric_model(
    type="custom_atmosphere",

    # Leaving the pressure field as None means we want the International Standard Atmosphere’s pressure profile to be used
    pressure = None,

    # The same is done with temperature set to None
    # The temperature field can be set as a constant K profile (example: 300)
    temperature = None,

    # Wind Speed East
    # an array containing values (x, y) where x = altitude (m) & y = windspeed (m/s)
    # positve y = East and negative y = West
    wind_u = [(0, 5.36)],

    # Wind Speed North
    # an array containing values (x, y) where x = altitude (m) & y = windspeed (m/s)
    # positive y = North and negative y = South
    wind_v = [(0, -2)],
    )


In [ ]:
# Retrieve all information about the Environment object
env.all_info()


### Creating a Motor




In [ ]:
# Define a SolidMotor object with specific parameters
PRO75L645 = SolidMotor(
    # Correct
    thrust_source="Cesaroni_3419L645-P.eng", # Specify the .eng file containing thrust data
    # Correct
    dry_mass=1.607, # Dry mass of the motor in kilograms
    # Assumption - Sean Reynolds Number
    dry_inertia=(0.039, 0.039, 0.0015), # Dry mass moments of inertia in kg*m^2
    # Assumption
    nozzle_radius = 37.5 / 1000, # Radius of the nozzle exit in meters
    # Correct
    grain_number=3, # Number of propellant grains
    # Assumption - Sean Bourke
    grain_density=1815, # Density of the propellant in kg/m^3
    # Assumption
    grain_outer_radius = 37.5 / 1000, # Outer radius of the propellant grains in meters (37.5)
    # Assumption
    grain_initial_inner_radius=29 / 1000, # Initial inner radius of the propellant grains in meters
    # Assumption - prolly important
    grain_initial_height=157 / 1000, # Initial height of the propellant grains in meters
    # Assumption - prolly important
    grain_separation=5 / 1000, # Separation between propellant grains in meters
    #Assumption - prolly important
    grains_center_of_mass_position=0.343, # Position of the center of mass of the propellant grains
    # Assumption
    center_of_dry_mass_position=0.231, # Position of the center of mass of the dry motor
    # Correct
    nozzle_position = 0.0, # Position of the nozzle relative to the combustion chamber
    # Correct
    burn_time=5.3, # Burn time of the motor in seconds

    throat_radius=12.5 / 1000, # Radius of the motor throat in meters
    # Known
    coordinate_system_orientation="nozzle_to_combustion_chamber" # Orientation of the coordinate system -> EXTREMLY IMPORTANT
)


In [ ]:
# Retrieve all information about the SolidMotor object
PRO75L645.all_info()


### Creating a Rocket


In [ ]:
# Define a Rocket object with specific parameters
rocket = Rocket(
    radius=106 / 2000, # Radius of the rocket body in meters
    mass=5.544, # Total mass of the rocket in kilogr...ams
    # These need to be checked
    inertia=(3.8 , 3.8, 0.01), # Moments of inertia of the rocket in kg*m^2

    power_off_drag = "DragOffCSV.csv", # .csv file containing power-off drag curve data

    power_on_drag = "DragOnCSV.csv", # .csv file containing power-on drag curve data

    center_of_mass_without_motor=87.1 / 100, # Position of the center of mass of the rocket without the motor

    coordinate_system_orientation="tail_to_nose" # Orientation of the coordinate system -> EXTREMLY IMPORTANT -> MUST be relative to the coordinate system defined in the motor
)

# Set the rail buttons for the rocket
rail_buttons = rocket.set_rail_buttons(
    upper_button_position=0.618, # Position of the upper rail button relative to the rocket nose
    lower_button_position=0.230, # Position of the lower rail button relative to the rocket nose
    angular_position=45, # Angular position of the rail buttons in degrees
)


In [ ]:
# Add the motor object to the rocket object at a specific position
# Said position is EXTREMLY IMPORTANT
rocket.add_motor(PRO75L645, position=0)


### Adding Aerodynamic Surfaces


In [ ]:
# Add a nose cone to the rocket object with specific parameters
nose_cone = rocket.add_nose(length=0.410, kind="vonKarman", position=1.815)

# Add trapezoidal fins to the rocket with specific parameters
fin_set = rocket.add_trapezoidal_fins(
    n=3, # Number of fins
    root_chord=0.25, # Root chord length of the fins in meters
    tip_chord=0.112, # Tip chord length of the fins in meters
    span=0.152, # Span of the fins in meters
    position=0.345, # Position of the fins relative to the rocket nose
    cant_angle=0, # Cant angle of the fins in radians
    # airfoil=("AirfoilDegreesCSV.csv", "degrees"), # Airfoil profile of the fins -> .csv file and declare your unit of measurment (radians OR degrees)
)

# Add a tail to the rocket object with specific parameters
tail = rocket.add_tail(
    top_radius = 106 / 2000, # Top radius of the tail in meters
    bottom_radius=0.04, # Bottom radius of the tail in meters
    length=0.08, # Length of the tail in meters
    position=0.08 # Position of the tail relative to the rocket nose
)


In [ ]:
# Retrieve all information about the Rocket object
rocket.all_info()


### Adding Parachutes


In [ ]:
# Add a main parachute to the rocket with specific parameters
Main = rocket.add_parachute(
    "Main", # Name of the parachute
    cd_s = 1.0, # Drag coefficient of the parachute in m^2
    trigger = 200, # Trigger for parachute deployment can be EITHER an altitude in meters OR an event
    sampling_rate = 105, # Sampling rate for parachute simulation
    lag = 1.5, # Lag time for parachute deployment in seconds
    noise = (0, 8.3, 0.5), # Noise parameters for parachute simulation
)

# Add a drogue parachute to the rocket with specific parameters
Drogue = rocket.add_parachute(
    "Drogue", # Name of the parachute
    cd_s = 0.313, # Drag coefficient of the parachute in m^2
    trigger = "apogee", # Trigger for parachute deployment can be EITHER an altitude in meters OR an event
    sampling_rate = 105, # Sampling rate for parachute simulation
    lag = 1.5, # Lag time for parachute deployment in seconds
    noise = (0, 8.3, 0.5), # Noise parameters for parachute simulation
)


## Simulating a Flight


In [ ]:
# Create a Flight object named test_flight with specific parameters
test_flight = Flight(
    rocket=rocket, # Rocket object representing the rocket to be simulated
    environment=env, # Environment object representing the atmospheric and environmental conditions
    rail_length=4, # Length of the launch rail in meters
    inclination=85, # Inclination angle of the launch rail in degrees
    heading = 106 # Heading angle of the launch rail in degrees
)


## Analyzing the Results


In [ ]:
# Retrieve all information about the test_flight Flight object
test_flight.all_info()


In [ ]:
from google.colab import files

# Export the trajectory of the test_flight to a KML file
test_flight.export_kml(
    file_name="Trajectory.kml", # Specify the file name
    extrude=True, # Enable extrusion for 3D visualization
    altitude_mode="relative_to_ground", # Specify the altitude mode
)

# Download the KML file for use in Google Earth
files.download('Trajectory.kml')
